In [2]:
!pip install river

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 65.9 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [46]:


from river.datasets import Phishing
import pandas as pd
import numpy as np
from utils import plot_drift
from novelty import ephemeral, persistant, periodic, degenerate

from river import datasets
from river import ensemble
from river import evaluate
from river import linear_model
from river import metrics
from river import optim
from river import preprocessing


from drift import detect_drift

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
phishing = Phishing()
list(phishing.take(2))

[({'empty_server_form_handler': 0.0,
   'popup_window': 0.0,
   'https': 0.0,
   'request_from_other_domain': 0.0,
   'anchor_from_other_domain': 0.0,
   'is_popular': 0.5,
   'long_url': 1.0,
   'age_of_domain': 1,
   'ip_in_url': 1},
  True),
 ({'empty_server_form_handler': 1.0,
   'popup_window': 0.0,
   'https': 0.5,
   'request_from_other_domain': 0.5,
   'anchor_from_other_domain': 0.0,
   'is_popular': 0.5,
   'long_url': 0.0,
   'age_of_domain': 1,
   'ip_in_url': 0},
  True)]

In [8]:
def dataset_to_df(dataset, count=None):
    if not count:
        count = len(list(dataset))
    data_list = [{**x, 'value': y} for x, y in dataset.take(count)]
    return pd.DataFrame(data_list)

def df_to_dataset(df):
    df_nonvalue = df.loc[:, df.columns != 'value']
    df_value = df['value']
    return list(zip(df_nonvalue.to_dict(orient='records'), df_value.to_list()))

In [9]:
df = dataset_to_df(phishing)
df

,empty_server_form_handler,popup_window,https,request_from_other_domain,anchor_from_other_domain,is_popular,long_url,age_of_domain,ip_in_url,value
0,0.0,0.0,0.0,0.0,0.0,0.5,1.0,1,1,True
1,1.0,0.0,0.5,0.5,0.0,0.5,0.0,1,0,True
2,0.0,0.0,1.0,0.0,0.5,0.5,0.0,1,0,True
3,0.0,0.0,1.0,0.0,0.0,1.0,0.5,0,0,True
4,1.0,0.0,0.5,1.0,0.0,0.5,0.5,1,0,False
...,...,...,...,...,...,...,...,...,...,...
1245,0.0,0.0,0.0,0.0,0.0,0.0,0.5,1,0,True
1246,0.0,0.5,1.0,0.5,0.0,0.5,0.5,1,0,False
1247,0.0,0.5,0.0,0.0,0.0,0.5,0.0,0,0,True
1248,0.5,0.5,1.0,0.5,0.5,0.5,0.0,1,0,True


In [10]:
def eval(dataset):
    metric = metrics.F1()

    model_ADWIN = ensemble.ADWINBaggingClassifier(
        model=(
            preprocessing.StandardScaler() |
            linear_model.LogisticRegression()
        ),
        n_models=10,
        seed=42
    )

    model_plain = (
        preprocessing.StandardScaler() |
        linear_model.LogisticRegression()
    )

    ADWIN_result = evaluate.progressive_val_score(
        dataset,
        model_ADWIN,
        metric,)

    print(f"ADWIN: {ADWIN_result}")

    plain_result = evaluate.progressive_val_score(
        dataset,
        model_plain,
        metric,)

    print(f"plain: {plain_result}")

Ephemeral Novelty

In [11]:
print("Pre-Novelty Insertion")
eval(phishing)

print()
print("Post-Novelty Insertion")
df_ephemeral = ephemeral(df, 'value', 200, 500, lambda x: not x)
eval(df_to_dataset(df_ephemeral))

Pre-Novelty Insertion
ADWIN: F1: 87.97%
plain: F1: 87.97%

Post-Novelty Insertion
ADWIN: F1: 77.58%
plain: F1: 76.72%


In [12]:
def eval_Ada(dataset):
    model_Ada = ensemble.AdaBoostClassifier(
        model=(
            preprocessing.StandardScaler() |
            linear_model.LogisticRegression()
        ),
        n_models=5,

    )

    model_plain = (
        preprocessing.StandardScaler() |
        linear_model.LogisticRegression()
    )

    metric = metrics.F1()

    Ada_result = evaluate.progressive_val_score(
        dataset,
        model_Ada,
        metric,)

    print(f"Ada: {Ada_result}")


    plain_result = evaluate.progressive_val_score(
        dataset,
        model_plain,
        metric,)

    print(f"plain: {plain_result}")

In [13]:
print("Pre-Novelty")
eval_Ada(phishing)

print()
print("Post-Novelty Insertion")
eval_Ada(df_to_dataset(df_ephemeral))

Pre-Novelty
Ada: F1: 87.04%
plain: F1: 87.51%

Post-Novelty Insertion
Ada: F1: 86.39%
plain: F1: 81.05%


Persistant Novelty

In [33]:
print("Pre-Novelty Insertion")
eval(phishing)

print()
print("Post-Novelty Insertion")
df_persistant = persistant(df, 'value', 600, lambda x: not x) # insert persistant novelty
eval(df_to_dataset(df_persistant))

Pre-Novelty Insertion
ADWIN: F1: 87.97%
plain: F1: 87.97%

Post-Novelty Insertion
ADWIN: F1: 76.50%
plain: F1: 76.31%


In [34]:
def eval_Ada(dataset):
    model_Ada = ensemble.AdaBoostClassifier(
        model=(
            preprocessing.StandardScaler() |
            linear_model.LogisticRegression()
        ),
        n_models=5,

    )

    model_plain = (
        preprocessing.StandardScaler() |
        linear_model.LogisticRegression()
    )

    metric = metrics.F1()

    Ada_result = evaluate.progressive_val_score(
        dataset,
        model_Ada,
        metric,)

    print(f"Ada: {Ada_result}")


    plain_result = evaluate.progressive_val_score(
        dataset,
        model_plain,
        metric,)

    print(f"plain: {plain_result}")

In [35]:
print("Pre-Novelty")
eval_Ada(phishing)

print()
print("Post-Novelty Insertion")
eval_Ada(df_to_dataset(df_persistant))

Pre-Novelty
Ada: F1: 87.74%
plain: F1: 87.86%

Post-Novelty Insertion
Ada: F1: 86.93%
plain: F1: 81.63%


Periodic Novelty

In [39]:
print("Pre-Novelty Insertion")
eval(phishing)

print()
print("Post-Novelty Insertion")
df_periodic = periodic(df, 'value', location=200, duration=10, period=100, func=lambda x: not x) # insert periodic

eval(df_to_dataset(df_periodic))

Pre-Novelty Insertion
ADWIN: F1: 87.97%
plain: F1: 87.97%

Post-Novelty Insertion
ADWIN: F1: 79.51%
plain: F1: 79.66%


In [40]:
def eval_Ada(dataset):
    model_Ada = ensemble.AdaBoostClassifier(
        model=(
            preprocessing.StandardScaler() |
            linear_model.LogisticRegression()
        ),
        n_models=5,

    )

    model_plain = (
        preprocessing.StandardScaler() |
        linear_model.LogisticRegression()
    )

    metric = metrics.F1()

    Ada_result = evaluate.progressive_val_score(
        dataset,
        model_Ada,
        metric,)

    print(f"Ada: {Ada_result}")


    plain_result = evaluate.progressive_val_score(
        dataset,
        model_plain,
        metric,)

    print(f"plain: {plain_result}")

In [41]:
print("Pre-Novelty")
eval_Ada(phishing)

print()
print("Post-Novelty Insertion")
eval_Ada(df_to_dataset(df_periodic))

Pre-Novelty
Ada: F1: 87.35%
plain: F1: 87.66%

Post-Novelty Insertion
Ada: F1: 77.87%
plain: F1: 78.84%


Degenerate Novelty

In [68]:
print("Pre-Novelty Insertion")
eval(phishing)

print()
print("Post-Novelty Insertion")
df_degenerate = degenerate(df, 'value', start=800, duration=100, func=lambda x: x + np.random.randint(-1000, 1000))

eval(df_to_dataset(df_degenerate))

Pre-Novelty Insertion
ADWIN: F1: 87.97%
plain: F1: 87.97%

Post-Novelty Insertion


/content/novelty.py:70: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ 424 -192 -444 -345  202  248  363  349  -68 -607   28  119  893 -166
 -875  897   90  -65  258  652  890  120 -818  148  244  787 -313 -303
  923  597 -159  385  655 -524 -937  459   15   83  276 -553  429  518
  370  887  340 -218  824 -100   91  452  161  807 -485  337  739  295
 -140 -910 -635  248 -283  770 -519  -77  899 -288  582 -278 -830  675
 -877  880  377 -789  160 -306 -568 -655   52  593  900  591  245 -972
 -667 -687 -900  727  -30 -242  975   97  553 -163 -347 -805  648 -929
  516  255   18]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  data.loc[start:end, feature_name] = data.loc[start:end, feature_name].apply(func)


ADWIN: F1: 80.40%
plain: F1: 78.77%


In [69]:
def eval_Ada(dataset):
    model_Ada = ensemble.AdaBoostClassifier(
        model=(
            preprocessing.StandardScaler() |
            linear_model.LogisticRegression()
        ),
        n_models=5,

    )

    model_plain = (
        preprocessing.StandardScaler() |
        linear_model.LogisticRegression()
    )

    metric = metrics.F1()

    Ada_result = evaluate.progressive_val_score(
        dataset,
        model_Ada,
        metric,)

    print(f"Ada: {Ada_result}")


    plain_result = evaluate.progressive_val_score(
        dataset,
        model_plain,
        metric,)

    print(f"plain: {plain_result}")

In [70]:
print("Pre-Novelty")
eval_Ada(phishing)

print()
print("Post-Novelty Insertion")
eval_Ada(df_to_dataset(df_degenerate))

Pre-Novelty
Ada: F1: 87.29%
plain: F1: 87.64%

Post-Novelty Insertion
Ada: F1: 79.44%
plain: F1: 78.28%
